In [1]:
import pandas as pd
import os
import pickle
from random import choice
from datetime import datetime
from sys import getsizeof

In [2]:
# paths
directory = '/home/jake/Documents/ML_BGP/results'
snapshot_dir = '/home/jake/Documents/ML_BGP'

In [3]:
# initialize variables
prefix_dict = dict()

In [4]:
# constants
ACTION_IDX = 2
PREFIX_IDX = 5
TIME_IDX = 1
VPASN_IDX = 4

In [5]:
getsizeof('192.168.0.0/24')


63

In [6]:
# datetime.utcfromtimestamp(start)
class record_new:
    def __init__(self, init_state, time):
        self.last_state = init_state
        self.timestamps = []
        self.timestamp = float(time)
    def update(self, state, time):
        if state == 'W' and (self.last_state == 'B' or self.last_state == 'A'):
            self.last_state = 'W'
            self.timestamps.append([self.timestamp, float(time)])
        elif state == 'A' and self.last_state == 'W':
            self.last_state = 'A'
            self.timestamp = float(time)

In [7]:
# find a rib file (snapshot) to set up initial state for all prefixes
for filename in os.listdir(snapshot_dir):
    if 'rib' in filename:
        with open(filename, 'r') as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip('\n').split('|')
                key = (line[VPASN_IDX], line[PREFIX_IDX])
                prefix_dict[key] = record_new(line[ACTION_IDX], line[TIME_IDX])
        break

In [8]:
# scan through available updates and record all uptimes
update_dir_list = sorted(os.listdir(directory))
for filename in update_dir_list:
    if filename.endswith(".txt"):
        with open('./results/' + filename, 'r') as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip('\n').split('|')
                key = (line[VPASN_IDX], line[PREFIX_IDX])
                if key not in prefix_dict and line[ACTION_IDX] == 'A':
                    prefix_dict[key] = record_new(line[ACTION_IDX], line[TIME_IDX])
                elif key in prefix_dict:
                    prefix_dict[key].update(line[ACTION_IDX], line[TIME_IDX])

In [9]:
print(len(prefix_dict))

29373890


In [22]:
def ip_to_binary(ip):
    octet_list_int = ip.split(".")
    octet_list_bin = [format(int(i), '08b') for i in octet_list_int]
    binary = ("").join(octet_list_bin)
    return binary
def get_addr_network(address, net_size):
    #Convert ip address to 32 bit binary
    ip_bin = ip_to_binary(address)
    #Extract Network ID from 32 binary
    network = ip_bin[0:32-(32-net_size)]    
    return network
def ip_in_prefix(ip_address, prefix):
    #CIDR based separation of address and network size
    [prefix_address, net_size] = prefix.split("/")
    #Convert string to int
    net_size = int(net_size)
    #Get the network ID of both prefix and ip based net size
    prefix_network = get_addr_network(prefix_address, net_size)
    ip_network = get_addr_network(ip_address, net_size)
    return ip_network == prefix_network
def get_prefix_in_decimal(prefix):
    [prefix_address, net_size] = prefix.split("/")
    prefix_network = get_addr_network(prefix_address, int(net_size))
    return str(int(prefix_network,2))+'-'+net_size

In [28]:
import csv
back_up_path = './big_csv.csv'
with open(back_up_path, 'w') as f:
    csv_writer = csv.writer(f, delimiter=',')
    for item in prefix_dict.items():
        if len(item[1].timestamps) > 0:
            counter = 0
            for each_pair in item[1].timestamps:
                row = [item[0][0],item[0][1], each_pair[0], each_pair[1],each_pair[1] - float(each_pair[0]), counter]
                csv_writer.writerow(row)
                counter += 1

In [29]:
back_up_path = './stable_link.csv'
with open(back_up_path, 'w') as f:
    csv_writer = csv.writer(f, delimiter=',')
    for item in prefix_dict.items():
        if len(item[1].timestamps) == 0:
                row = [item[0][0],item[0][1]]
                csv_writer.writerow(row)

In [ ]:
data = sorted(prefix_dict.items(), key = lambda item: len(item[1].uptimes) , reverse = True)

In [ ]:
for i in range(len(data)):
    if (len(data[i][1].uptimes)) == 0:
        print(i)
        break

In [30]:
with open('save.pickle', 'wb') as f:
    pickle.dump(prefix_dict ,f)

In [32]:
import dask.dataframe as dd
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard
# http://localhost:8787/status

In [36]:
df = dd.read_csv('./stable_link.csv')
dd.to_parquet(df,'./large_parquets/stable_link')

In [37]:
df = dd.read_csv('./big_csv.csv')
dd.to_parquet(df,'./large_parquets/big_csv')

In [38]:
df = dd.read_parquet('./large_parquets/big_csv')
df.head()

,ASN,Prefix,Start Time,End Time,Uptime,Counter
0,57463,1.0.0.0/24,1.593562e+09,1.593627e+09,65705.801747,0
1,18106,1.0.0.0/24,1.593562e+09,1.593984e+09,422868.447900,0
2,57866,1.0.0.0/24,1.593562e+09,1.593594e+09,32367.444219,0
3,57866,1.0.0.0/24,1.593594e+09,1.594283e+09,688706.428775,1
4,53767,1.0.4.0/22,1.593562e+09,1.594479e+09,917185.474684,0


In [46]:
counter_df = df.groupby(['ASN','Prefix']).count()